In [14]:
### Add finale labels to features

# dependecies
import pandas as pd
import numpy as np

In [3]:
# data
read_path_labels = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\Model Building\MW_Probes_final.csv"
read_path_features = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\features_final.csv"
save_path = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\Model Building\features_with_labels.csv"

# read Data
df_labels = pd.read_csv(read_path_labels)
df_features = pd.read_csv(read_path_features)

df_labels.replace("P04","4", inplace= True)

In [4]:
# exclude participants
excluded_participants = [ str(i) for i in [2, 8, 11, 12, 13, 24, 25, 27, 30, 31, 38, 41, 43, 47, 50, 52, 54, 59, 63, 69, 70, 75, 84, 89, 91, 101, 102, 103, 104, 105, 999]]
df_labels = df_labels.loc[~df_labels["SubjectName"].isin(excluded_participants)]
df_labels["SubjectName"].unique()

array(['61', '62', '64', '65', '66', '67', '1', '3', '4', '5', '6', '7',
       '9', '10', '14', '15', '16', '17', '18', '19', '22', '20', '21',
       '23', '26', '28', '29', '32', '33', '34', '35', '68', '71', '72',
       '73', '74', '36', '37', '39', '40', '42', '44', '45', '46', '49',
       '48', '51', '53', '76', '77', '78', '79', '80', '81', '83', '55',
       '56', '57', '85', '86', '87', '88', '90', '92', '93', '58', '60',
       '94', '95', '97'], dtype=object)

In [5]:
# make the dataframes more similar, that we can merge
df_features["Stimulus"] = df_features["Stimulus"].str.replace("vid", " ").astype("int")

df_labels = df_labels.rename(columns={"SubjectName": "Participant"})
df_features = df_features.rename(columns = {"Stimulus": "Probe"})
df_labels["Participant"] = df_labels["Participant"].astype("int")
df_features["Participant"] = df_features["Participant"].astype("int")

In [6]:
df_labels

,Participant,Probe,Timestamps,Interaction,Content,Awareness,Awareness_cat,Awareness_text,Content_cat,Content_text,Content_Probe
0,61,1,3:20,0,Wie gut ich die Vorlesung verstehe (für bewert...,"Ich habe nicht bemerkt, dass meine Gedanken ab...",1,unaware,3.0,Lecture Comprehension (TRI),TRI
1,61,2,7:24,0,zurückliegender Mathematik-Unterricht während ...,"Ich habe bemerkt, dass meine Gedanken abschwei...",2,aware,9.0,Lecture triggered (TUT),TUT
2,61,3,8:23,0,"Ich war bei der Sache, folgte der Vorlesung (f...",Trifft nicht zu,3,NaN,1.0,On task,On Task
3,61,4,10:35,0,Wie gut ich die Vorlesung verstehe (für bewert...,Trifft nicht zu,3,NaN,3.0,Lecture Comprehension (TRI),TRI
4,61,5,16:46,0,Aktueller Zustand (für Gedanken über den aktue...,"Ich habe bemerkt, dass meine Gedanken abschwei...",2,aware,6.0,Current state (TUT),TUT
...,...,...,...,...,...,...,...,...,...,...,...
1345,97,11,39:25,0,Aktueller Zustand (für Gedanken über den aktue...,"Ich habe bemerkt, dass meine Gedanken abschwei...",2,aware,6.0,Current state (TUT),TUT
1346,97,12,44:22,0,Alltägliche persönliche Anliegen (für Gedanken...,"Ich habe bemerkt, dass meine Gedanken abschwei...",2,aware,4.0,Personal matters (TUT),TUT
1347,97,13,47:45,0,"Ich war bei der Sache, folgte der Vorlesung (f...",Trifft nicht zu,3,NaN,1.0,On task,On Task
1348,97,14,51:25,0,Aktueller Zustand (für Gedanken über den aktue...,Trifft nicht zu,3,NaN,6.0,Current state (TUT),TUT


In [26]:
## wie bekomme ich die nan values???
(df_labels['Awareness_all'] == np.nan).unique()

array([False])

In [27]:
# to get the two categories task-related and task unrelated thoughts if the content probe is aslo in this category
# categories: ['task-related', 'aware', 'unaware', nan]
df_labels['Awareness_all'] = df_labels['Awareness_text']
df_labels.loc[df_labels['Content_Probe']!= "TUT", "Awareness_all"] = "task-related"

In [28]:
# we want to merge only the necessary columns
df_labels_necessary = df_labels[["Participant", "Probe", "Awareness_all"]]
df_labels_necessary

,Participant,Probe,Awareness_all
0,61,1,task-related
1,61,2,aware
2,61,3,task-related
3,61,4,task-related
4,61,5,aware
...,...,...,...
1345,97,11,aware
1346,97,12,aware
1347,97,13,task-related
1348,97,14,NaN


In [39]:
# give them categories instead of numbers
# categories: ['task-related' == 1 , 'aware' == 2, 'unaware' == 3 , nan == delete the row]

# create TRT, TRI, TUT categorization
# create categories for content probes:
list_cont= []
for index, row in df_labels_necessary.iterrows():
    if row['Awareness_all'] == 'task-related':
        list_cont.append(1)
    elif row['Awareness_all'] == 'aware':
        list_cont.append(2)
    elif row['Awareness_all'] == 'unaware':
        list_cont.append(3)
    else:
        # delete all nan values
        df_labels_necessary = df_labels_necessary.drop(index)
  
 
df_labels_necessary['Awareness_all_cat'] = list_cont
df_labels_necessary


,Participant,Probe,Awareness_all,Awareness_all_cat
0,61,1,task-related,1
1,61,2,aware,2
2,61,3,task-related,1
3,61,4,task-related,1
4,61,5,aware,2
...,...,...,...,...
1344,97,10,aware,2
1345,97,11,aware,2
1346,97,12,aware,2
1347,97,13,task-related,1


In [40]:
# merging and saving as csv
df_features_with_label = pd.merge(df_features, df_labels_necessary, on = ["Participant","Probe"])
df_features_with_label.to_csv(save_path)